<a href="https://colab.research.google.com/github/fmarcelneves/Twitter-Network-Analytics/blob/main/Machine_learning_model_political_theme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Portuguese Tweets for Sentiment Analysis</h1>

## Install package

In [ ]:
!pip install xgboost

## Load packages

In [ ]:
import os
import pandas as pd
import numpy as np
import random
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
# from nltk.tokenize import sent_tokenize (Tokenization)
from nltk.probability import FreqDist
from nltk.metrics import ConfusionMatrix
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
import  xgboost as xgb

In [ ]:
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
%%capture
!pip install plotly --upgrade
from plotly import __version__
print (__version__)
import plotly.express as px
import plotly as py
#import plotly.figure_factory as ff
#import cufflinks as cf
from plotly.offline import iplot, init_notebook_mode
# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
import plotly.io as pio
pio.templates.default = "plotly_white"

## Load and concatenate dataset

<h4>Load and prepare</h4>

In [ ]:
TweetsNeutralHash = 'https://raw.githubusercontent.com/fmarcelneves/Twitter-Network-Analytics/main/TweetsNeutralHash.csv'
TweetsNeutralNews = 'https://raw.githubusercontent.com/fmarcelneves/Twitter-Network-Analytics/main/TweetsNeutralNews.csv'
TweetsWithTheme = 'https://raw.githubusercontent.com/fmarcelneves/Twitter-Network-Analytics/main/TweetsWithTheme.csv'
#Train3Classes = 'https://raw.githubusercontent.com/fmarcelneves/Twitter-Network-Analytics/main/Train3Classes.csv'
NoThemeTweets = 'https://raw.githubusercontent.com/fmarcelneves/Twitter-Network-Analytics/main/NoThemeTweets.csv'

In [ ]:
NeutralHash = pd.read_csv(TweetsNeutralHash, encoding='utf-8')
NeutralNews = pd.read_csv(TweetsNeutralNews, encoding='utf-8')
Political_Neg_Pos = pd.read_csv(TweetsWithTheme, encoding='utf-8')
#Classes = pd.read_csv(Train3Classes, encoding='utf-8', sep=';')
NoThemeTweets_Pos = pd.read_csv(NoThemeTweets, encoding='latin-1', sep=';')

Neutral

In [ ]:
NeutralHash  = NeutralHash[['tweet_text', 'sentiment']]
NeutralHash.columns = ['texto', 'classificacao']

In [ ]:
NeutralHash.head()

,texto,classificacao
0,@dilsonramoslima #Fato Acho que o Roger é um b...,Neutro
1,#NOVIDADE! @LATAM_BRA acaba de anunciar novo v...,Neutro
2,Quando tem #novidade😆 tem @novafm103 na área! ...,Neutro
3,@RiodeNojeira #Novidade Taí o sucesso dos filh...,Neutro
4,"[Livro/Novidades] Segredos, uma história de Lu...",Neutro


In [ ]:
NeutralHash = NeutralHash.drop_duplicates().reset_index(drop=True)

In [ ]:
NeutralHash.shape

(15515, 2)

In [ ]:
NeutralHash = NeutralHash.sample(8000) #14000 #8000

In [ ]:
NeutralHash.shape

(8000, 2)

In [ ]:
NeutralNews  = NeutralNews[['tweet_text', 'sentiment']]
NeutralNews.columns = ['texto', 'classificacao']

In [ ]:
NeutralNews = NeutralNews.drop_duplicates().reset_index(drop=True)

In [ ]:
NeutralNews.shape

(37509, 2)

In [ ]:
NeutralNews = NeutralNews.sample(20000) #14000 #20000

In [ ]:
NeutralNews.shape

(20000, 2)

In [ ]:
#NeutralHash+NeutralNews = Neutral

Positive 

In [ ]:
Political_Neg_Pos  = Political_Neg_Pos[['tweet_text', 'sentiment']]
Political_Neg_Pos.columns = ['texto', 'classificacao']

In [ ]:
Political_Neg_Pos.shape

(61591, 2)

In [ ]:
Political_Neg_Pos = Political_Neg_Pos.drop_duplicates().reset_index(drop=True)

In [ ]:
Political_Neg_Pos.shape

(61283, 2)

In [ ]:
Political_Neg_Pos.loc[Political_Neg_Pos['classificacao'] == 'Positivo'].shape

(32566, 2)

In [ ]:
Political_Pos = Political_Neg_Pos.loc[Political_Neg_Pos['classificacao'] == 'Positivo'].sample(20000) #18000 #20000 #14000

In [ ]:
Political_Pos.shape

(20000, 2)

In [ ]:
NoThemeTweets_Pos.columns = ['texto', 'classificacao']

In [ ]:
NoThemeTweets_Pos.head()

,texto,classificacao
0,@Tixaa23 14 para eu ir :),Positivo
1,@drexalvarez O meu like eu já dei na época :),Positivo
2,Eu só queria conseguir comer alguma coisa pra ...,Positivo
3,:D que lindo dia !,Positivo
4,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",Positivo


In [ ]:
NoThemeTweets_Pos = NoThemeTweets_Pos.drop_duplicates().reset_index(drop=True)

In [ ]:
NoThemeTweets_Pos.shape

(19983, 2)

In [ ]:
NoTheme_Pos = NoThemeTweets_Pos[NoThemeTweets_Pos['classificacao'] == 'Positivo'].sample(8000)

In [ ]:
NoTheme_Pos.shape

(8000, 2)

In [ ]:
#from IPython.display import HTML
#HTML(Political_Pos.to_html())

In [ ]:
#Classes_Pos  = Classes[['tweet_text', 'sentiment']]
#Classes_Pos.columns = ['texto', 'classificacao']

In [ ]:
#Classes_Pos['classificacao'] = Classes_Pos['classificacao'].map({0: 'Negativo', 1: 'Positivo', 2: 'Neutro'})

In [ ]:
#Classes_Pos = Classes_Pos.loc[Classes_Pos['classificacao'] == 'Positivo'].sample(8000) #10000 #14000 #8000

In [ ]:
#Classes_Pos.shape

In [ ]:
#Political_Pos+NoTheme_Pos = Positive

Negative

In [ ]:
Political_Neg_Pos = Political_Neg_Pos.drop_duplicates().reset_index(drop=True)

In [ ]:
Political_Neg_Pos.loc[Political_Neg_Pos['classificacao'] == 'Negativo'].shape

(28717, 2)

In [ ]:
Political_Neg = Political_Neg_Pos.loc[Political_Neg_Pos['classificacao'] == 'Negativo'].sample(28000)

In [ ]:
Political_Neg.shape

(28000, 2)

In [ ]:
dataset = pd.concat([NeutralHash, NeutralNews, Political_Pos, NoTheme_Pos, Political_Neg]).drop_duplicates().reset_index(drop=True)
dataset

,texto,classificacao
0,Esta é pra você amigo! No meio da dificuldade ...,Neutro
1,Conheça histórias de pessoas que encontraram o...,Neutro
2,"De cabelo em pé com essa semana 🤪, doideira, m...",Neutro
3,"@junioitatiaia Insônia, Amigo! Não desejo a ni...",Neutro
4,"""Deus é bom o tempo TODO. O tempo TODO Ele é b...",Neutro
...,...,...
83925,@udangerz Não faz issooooo! Facismo naaaaao! A...,Negativo
83926,esses votos no ciro mudaria tanta coisa :( htt...,Negativo
83927,"Oi Marina, não sei nem como falar isso, mas......",Negativo
83928,@raulnascimentoo @conexaopolitica @jairbolsona...,Negativo


In [ ]:
dataset.shape

(83930, 2)

In [ ]:
#dataset = pd.concat([NeutralHash, NeutralNews, Political_Pos, Political_Neg]).drop_duplicates().reset_index(drop=True)
#dataset

In [ ]:
#dataset = pd.concat([NeutralNews, Political_Pos, Political_Neg]).drop_duplicates().reset_index(drop=True)
#dataset

## Exploratory Analysis

In [ ]:
dataset.sample(5)

,texto,classificacao
6728,Engraçado é que #TODOS da #esquerda chamam a #...,Neutro
70810,@Crunchyroll_PT quando vocês vão começar a col...,Negativo
82426,@ejessicabarreto Em compensação os candidatos ...,Negativo
22947,Ataque com faca deixa 2 mortos e 1 ferido na F...,Neutro
71642,alckmin meu vovozinho não fica triste estou co...,Negativo


In [ ]:
series = dataset['classificacao'].value_counts()
series

Negativo    28000
Neutro      27994
Positivo    27936
Name: classificacao, dtype: int64

In [ ]:
pd.options.plotting.backend = "plotly"

In [ ]:
series.plot(kind='bar', title='Número de tweets classificados para cada sentimento considerado', 
labels={"value": "Frequências", "index": "Sentimentos"}, color = ['Negativo', 'Neutro', 'Positivo'], 
color_discrete_sequence =['#EF553B', '#00CC96', '#636EFA'])

## Text Preprocessing

<h4>Noise Removal</h4>

Remove urls from text (http(s), www)

In [ ]:
def _remove_url(data):
    ls = []
    words = ''
    regexp1 = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    regexp2 = re.compile('www?.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    
    for line in data:
        urls = regexp1.findall(line)

        for u in urls:
            line = line.replace(u, ' ')

        urls = regexp2.findall(line)

        for u in urls:
            line = line.replace(u, ' ')
            
        ls.append(line)
    return ls

In [ ]:
dataset['texto'] = _remove_url(dataset['texto'])

Remove regex patterns (hashtags, notations)

In [ ]:
def _remove_regex(data, regex_pattern):
    ls = []
    words = ''
    
    for line in data:
        matches = re.finditer(regex_pattern, line)
        
        for m in matches: 
            line = re.sub(m.group().strip(), '', line)

        ls.append(line)

    return ls

In [ ]:
# hashtags
regex_pattern = '#[\w]*'
dataset['texto'] = _remove_regex(dataset['texto'], regex_pattern)

In [ ]:
# notations
regex_pattern = '@[\w]*'
dataset['texto'] = _remove_regex(dataset['texto'], regex_pattern)

In [ ]:
# rt
regex_pattern = 'RT[\w]*'
dataset['texto'] = _remove_regex(dataset['texto'], regex_pattern)

In [ ]:
#numbers 
regex_pattern = '/\d/'
dataset['texto'] = _remove_regex(dataset['texto'], regex_pattern)

In [ ]:
 emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

In [ ]:
# emoji pattern 
regex_pattern = emoji_pattern 
dataset['texto'] = _remove_regex(dataset['texto'], regex_pattern)

In [ ]:
# check data
for i in range(0, 5):
    print(dataset['texto'][i])

Esta é pra você amigo! No meio da dificuldade encontra-se a oportunidade. Albert Einstein Pare, respire e pense! • • • • •     …  
Conheça histórias de pessoas que encontraram o grande amor nas ruas. Não perca o Repórter Record Investigação nesta segunda-feira (10), às 22h30, na tela da Record TV Belém!  •••       
De cabelo em pé com essa semana 🤪, doideira, mas vamos pra cima 🤨  livedogemino     em Agência ALL G  
 Insônia, Amigo! Não desejo a ninguém....
"Deus é bom o tempo TODO. O tempo TODO Ele é bom!! "  Escute a voz que fala dentro de ti, é Ele falando contigo. 


replace emoticons ":)) :) :D :(" to positive_emoticon or negative_emoticon or neutral_emoticon

In [ ]:
def _replace_emoticons(data, emoticon_list):
    ls = []

    for line in data:
        for exp in emoticon_list:
            line = line.replace(exp, emoticon_list[exp])

        ls.append(line)

    return ls

In [ ]:
emoticon_list = {':))': 'emoticon_positivo', ':)': 'emoticon_positivo', ':D': 'emoticon_positivo', ':-)': 'emoticon_positivo',  ':(': 'emoticon_negativo', ':((': 'emoticon_negativo', ':-(' : 'emoticon_negativo', '8)': 'emoticon_neutro'}

In [ ]:
dataset['texto']= _replace_emoticons(dataset['texto'], emoticon_list)

In [ ]:
# check data
for i in range(0, 5):
    print(dataset['texto'][i])

Esta é pra você amigo! No meio da dificuldade encontra-se a oportunidade. Albert Einstein Pare, respire e pense! • • • • •     …  
Conheça histórias de pessoas que encontraram o grande amor nas ruas. Não perca o Repórter Record Investigação nesta segunda-feira (10), às 22h30, na tela da Record TV Belém!  •••       
De cabelo em pé com essa semana 🤪, doideira, mas vamos pra cima 🤨  livedogemino     em Agência ALL G  
 Insônia, Amigo! Não desejo a ninguém....
"Deus é bom o tempo TODO. O tempo TODO Ele é bom!! "  Escute a voz que fala dentro de ti, é Ele falando contigo. 


In [ ]:
nan_value = float("NaN")
dataset.replace("", nan_value, inplace=True)

<h4>Tokenize text with nltk</h4>

In [ ]:
def _tokenize_text(data):
    ls = []

    for line in data:
        tokens = wordpunct_tokenize(line)
        ls.append(tokens)

    return ls

In [ ]:
dataset_texto_tokens = _tokenize_text(dataset['texto'])

In [ ]:
# check data
for i in range(0, 5):
    print(dataset_texto_tokens[i])

['Esta', 'é', 'pra', 'você', 'amigo', '!', 'No', 'meio', 'da', 'dificuldade', 'encontra', '-', 'se', 'a', 'oportunidade', '.', 'Albert', 'Einstein', 'Pare', ',', 'respire', 'e', 'pense', '!', '•', '•', '•', '•', '•', '…']
['Conheça', 'histórias', 'de', 'pessoas', 'que', 'encontraram', 'o', 'grande', 'amor', 'nas', 'ruas', '.', 'Não', 'perca', 'o', 'Repórter', 'Record', 'Investigação', 'nesta', 'segunda', '-', 'feira', '(', '10', '),', 'às', '22h30', ',', 'na', 'tela', 'da', 'Record', 'TV', 'Belém', '!', '•••']
['De', 'cabelo', 'em', 'pé', 'com', 'essa', 'semana', '🤪,', 'doideira', ',', 'mas', 'vamos', 'pra', 'cima', '🤨', 'livedogemino', 'em', 'Agência', 'ALL', 'G']
['Insônia', ',', 'Amigo', '!', 'Não', 'desejo', 'a', 'ninguém', '....']
['"', 'Deus', 'é', 'bom', 'o', 'tempo', 'TODO', '.', 'O', 'tempo', 'TODO', 'Ele', 'é', 'bom', '!!', '"', 'Escute', 'a', 'voz', 'que', 'fala', 'dentro', 'de', 'ti', ',', 'é', 'Ele', 'falando', 'contigo', '.']


In [ ]:
#dataset_texto_tokens = list(filter(None, dataset_texto_tokens))

In [ ]:
# check data
for i in range(0, 5):
    print(dataset_texto_tokens[i])

['Esta', 'é', 'pra', 'você', 'amigo', '!', 'No', 'meio', 'da', 'dificuldade', 'encontra', '-', 'se', 'a', 'oportunidade', '.', 'Albert', 'Einstein', 'Pare', ',', 'respire', 'e', 'pense', '!', '•', '•', '•', '•', '•', '…']
['Conheça', 'histórias', 'de', 'pessoas', 'que', 'encontraram', 'o', 'grande', 'amor', 'nas', 'ruas', '.', 'Não', 'perca', 'o', 'Repórter', 'Record', 'Investigação', 'nesta', 'segunda', '-', 'feira', '(', '10', '),', 'às', '22h30', ',', 'na', 'tela', 'da', 'Record', 'TV', 'Belém', '!', '•••']
['De', 'cabelo', 'em', 'pé', 'com', 'essa', 'semana', '🤪,', 'doideira', ',', 'mas', 'vamos', 'pra', 'cima', '🤨', 'livedogemino', 'em', 'Agência', 'ALL', 'G']
['Insônia', ',', 'Amigo', '!', 'Não', 'desejo', 'a', 'ninguém', '....']
['"', 'Deus', 'é', 'bom', 'o', 'tempo', 'TODO', '.', 'O', 'tempo', 'TODO', 'Ele', 'é', 'bom', '!!', '"', 'Escute', 'a', 'voz', 'que', 'fala', 'dentro', 'de', 'ti', ',', 'é', 'Ele', 'falando', 'contigo', '.']


<h4>Object Standardization</h4>

In [ ]:
def _apply_standardization(tokens, std_list):
    ls = []

    for tk_line in tokens:
        new_tokens = []
        
        for word in tk_line:
            if word.lower() in std_list:
                word = std_list[word.lower()]
                
            new_tokens.append(word) 
            
        ls.append(new_tokens)

    return ls

In [ ]:
# create your own list
std_list = {'eh': 'é', 'vc': 'você', 'vcs': 'vocês','tb': 'também', 'tbm': 'também', 'obg': 'obrigado', 'gnt': 'gente', 'q': 'que', 'n': 'não', 'cmg': 'comigo', 'p': 'para', 'ta': 'está', 'to': 'estou', 'vdd': 'verdade'}

In [ ]:
dataset_texto_tokens = _apply_standardization(dataset_texto_tokens, std_list)

In [ ]:
# check data
print(dataset_texto_tokens[4], dataset_texto_tokens[1008])

['"', 'Deus', 'é', 'bom', 'o', 'tempo', 'TODO', '.', 'O', 'tempo', 'TODO', 'Ele', 'é', 'bom', '!!', '"', 'Escute', 'a', 'voz', 'que', 'fala', 'dentro', 'de', 'ti', ',', 'é', 'Ele', 'falando', 'contigo', '.'] ['Fique', 'por', 'dentro', 'das', 'vagas', 'de', 'emprego', 'que', 'o', 'PAT', 'separou', 'para', 'esta', 'sexta', '-', 'feira', '(', '31', ').', 'É', 'sua', 'oportunidade', '.', 'Boralá', '?']


Remove stopwords

In [ ]:
def _remove_stopwords(tokens, stopword_list):
    ls = []

    for tk_line in tokens:
        new_tokens = []
        
        for word in tk_line:
            if word.lower() not in stopword_list:
                new_tokens.append(word) 
            
        ls.append(new_tokens)
        
    return ls

In [ ]:
stopword_list = []

In [ ]:
# get nltk portuguese stopwords
nltk_stopwords = nltk.corpus.stopwords.words('portuguese')

In [ ]:
# You can also add stopwords manually instead of loading from the database. Generally, we add stopwords that belong to this context.
stopword_list.append('é')
stopword_list.append('vou')
stopword_list.append('que')
stopword_list.append('tão')
stopword_list.append('...')
stopword_list.append('«')
stopword_list.append('➔')
stopword_list.append('|')
stopword_list.append('!!')
stopword_list.append('!!!')
stopword_list.append('!!!!')
stopword_list.append('//')
stopword_list.append('/')
stopword_list.append('||') 
stopword_list.append(':')
stopword_list.append('-')
stopword_list.append(';')
stopword_list.append('»')
stopword_list.append('…')
stopword_list.append(')')
stopword_list.append('(')
stopword_list.append('uai') 
stopword_list.append(']')
stopword_list.append('[')
stopword_list.append('[]')
stopword_list.append('((')
stopword_list.append('))')
stopword_list.append(')!')
stopword_list.append(',')
stopword_list.append(', ')
stopword_list.append('.')
stopword_list.append('..')
stopword_list.append('...')
stopword_list.append('....')
stopword_list.append('•')
stopword_list.append('$')
stopword_list.append('%')
stopword_list.append('()')
stopword_list.append(').')
stopword_list.append('"')
stopword_list.append('”')
stopword_list.append('“')
stopword_list.append("'")
stopword_list.append('=')
stopword_list.append('&')
stopword_list.append('\u200d')
stopword_list.append('“”')
stopword_list.append('[]')
stopword_list.append('^^')
stopword_list.append('(((((')
stopword_list.append('(((')
stopword_list.append(')))')
stopword_list.append(')))))')
stopword_list.append('")')
stopword_list.append('("')
stopword_list.append('emoticon_positivo') 
stopword_list.append('emoticon_negativo')
stopword_list.append('emoticon_neutro')

<h5 style="color:red;">A series of pontuaction characterters were removed 
<b>
...? and ! can be important to express sentiment and were maintained.</h5>

In [ ]:
# join all stopwords
stopword_list.extend(nltk_stopwords)
#stopword_list.extend(custom_stopwords'\d+')

In [ ]:
# remove duplicate stopwords (unique list)
stopword_list = list(set(stopword_list))

In [ ]:
dataset_texto_tokens = _remove_stopwords(dataset_texto_tokens, stopword_list)

In [ ]:
# check data
for i in range(0, 5):
    print(dataset_texto_tokens[i])

['pra', 'amigo', '!', 'meio', 'dificuldade', 'encontra', 'oportunidade', 'Albert', 'Einstein', 'Pare', 'respire', 'pense', '!']
['Conheça', 'histórias', 'pessoas', 'encontraram', 'grande', 'amor', 'ruas', 'perca', 'Repórter', 'Record', 'Investigação', 'nesta', 'segunda', 'feira', '10', '),', '22h30', 'tela', 'Record', 'TV', 'Belém', '!', '•••']
['cabelo', 'pé', 'semana', '🤪,', 'doideira', 'vamos', 'pra', 'cima', '🤨', 'livedogemino', 'Agência', 'ALL', 'G']
['Insônia', 'Amigo', '!', 'desejo', 'ninguém']
['Deus', 'bom', 'tempo', 'TODO', 'tempo', 'TODO', 'bom', 'Escute', 'voz', 'fala', 'dentro', 'ti', 'falando', 'contigo']


Stemming (dimensionality reduction)

In [ ]:
def _apply_stemmer(tokens):
    ls = []
    stemmer = nltk.stem.RSLPStemmer()

    for tk_line in tokens:
        new_tokens = []
        
        for word in tk_line:
            word = str(stemmer.stem(word))
            new_tokens.append(word) 
            
        ls.append(new_tokens)
        
    return ls

In [ ]:
nltk.download('rslp')

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


True

In [ ]:
dataset_texto_stemm = _apply_stemmer(dataset_texto_tokens)

In [ ]:
# check data
for i in range(0, 5):
    print(dataset_texto_tokens[i])

['pra', 'amigo', '!', 'meio', 'dificuldade', 'encontra', 'oportunidade', 'Albert', 'Einstein', 'Pare', 'respire', 'pense', '!']
['Conheça', 'histórias', 'pessoas', 'encontraram', 'grande', 'amor', 'ruas', 'perca', 'Repórter', 'Record', 'Investigação', 'nesta', 'segunda', 'feira', '10', '),', '22h30', 'tela', 'Record', 'TV', 'Belém', '!', '•••']
['cabelo', 'pé', 'semana', '🤪,', 'doideira', 'vamos', 'pra', 'cima', '🤨', 'livedogemino', 'Agência', 'ALL', 'G']
['Insônia', 'Amigo', '!', 'desejo', 'ninguém']
['Deus', 'bom', 'tempo', 'TODO', 'tempo', 'TODO', 'bom', 'Escute', 'voz', 'fala', 'dentro', 'ti', 'falando', 'contigo']


<h4>Dataset preparation</h4>
Untokenize text (transform tokenized text into string list)

In [ ]:
def _untokenize_text(tokens):
    ls = []

    for tk_line in tokens:
        new_line = ''
        
        for word in tk_line:
            new_line += word + ' '
            
        ls.append(new_line)
        
    return ls

In [ ]:
dataset_texto_tokens = _untokenize_text(dataset_texto_tokens)

In [ ]:
dataset_texto_stemm = _untokenize_text(dataset_texto_stemm)

In [ ]:
# check data
for i in range(0, 5):
    print(dataset_texto_stemm[i])

pra amig ! mei dificuldad encontr oportun albert einstein par resp pens ! 
conheç histór pesso encontr grand am rua perc repórt record investig nest segund feir 10 ), 22h30 tel record tv belém ! ••• 
cabel pé seman 🤪, doid vam pra cim 🤨 livedogemin agênc all g 
insôn amig ! desej ninguém 
deu bom temp tod temp tod bom escut voz fal dentr ti fal contig 


## Implementing a Model


In [ ]:
X_tweets = np.array(dataset_texto_stemm,  dtype=object)
X_tweets

array(['pra amig ! mei dificuldad encontr oportun albert einstein par resp pens ! ',
       'conheç histór pesso encontr grand am rua perc repórt record investig nest segund feir 10 ), 22h30 tel record tv belém ! ••• ',
       'cabel pé seman 🤪, doid vam pra cim 🤨 livedogemin agênc all g ',
       ...,
       'oi marin sei fal voc inter menin ? ach atra s — spoil héter moment namor ',
       'magn malt fic ', 'horr né ? abraç 🤗 '], dtype=object)

In [ ]:
y_classes = dataset["classificacao"].values
y_classes

array(['Neutro', 'Neutro', 'Neutro', ..., 'Negativo', 'Negativo',
       'Negativo'], dtype=object)

In [ ]:
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.neural_network import MLPClassifier

Split data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tweets, y_classes, test_size=0.33, random_state=42)

## Random Forest

In [ ]:
model = RandomForestClassifier(random_state=42, n_jobs=-1)

In [ ]:
model_RF = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', model),
])

### Random forest model with Hold-out

In [ ]:
model_RF.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                

In [ ]:
predictions_RF = model_RF.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions_RF))

              precision    recall  f1-score   support

    Negativo       0.69      0.77      0.73      9256
      Neutro       0.86      0.81      0.83      9117
    Positivo       0.65      0.61      0.63      9244

    accuracy                           0.73     27617
   macro avg       0.73      0.73      0.73     27617
weighted avg       0.73      0.73      0.73     27617



In [ ]:
#for doc, prediction, y in zip(X_test[0:20], predictions_RF[0:20], y_test[0:20]):
#    print('%r => %s [%s]' % (doc, prediction, y))

## XGBoost 

In [ ]:
model_XGB = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', xgb.XGBClassifier(n_estimators=1000, n_jobs=-1, nthread=-1))
])

##XGBoost with Hold-out

In [ ]:
model_XGB.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=1, missing

In [ ]:
predictions_XGB = model_XGB.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions_XGB))

              precision    recall  f1-score   support

    Negativo       0.71      0.74      0.72      9227
      Neutro       0.78      0.85      0.81      9124
    Positivo       0.67      0.58      0.62      9344

    accuracy                           0.72     27695
   macro avg       0.72      0.72      0.72     27695
weighted avg       0.72      0.72      0.72     27695



## Neural Network - Multilayer Perceptron

<img src="https://st2.ning.com/topology/rest/1.0/file/get/1140047167?profile=original" width="500"/>

In [ ]:
model_NN = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('pca', TruncatedSVD(n_components=500)), 
    ('clf', MLPClassifier(hidden_layer_sizes=(100,50), #2 camadas escondidas sendo a primeira com 100 neurônios e a segunda com 50 neurônios 
                          random_state=42, 
                          activation='relu', #Try othe functions. Ex. tanh, relu...
                          max_iter=1000,
                          early_stopping=True))
])

### Neural network model with Hold-out

In [ ]:
model_NN.fit(X_train, y_train)
predictions_NN = model_NN.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions_NN))

              precision    recall  f1-score   support

    Negativo       0.72      0.75      0.73      9227
      Neutro       0.85      0.83      0.84      9124
    Positivo       0.65      0.65      0.65      9344

    accuracy                           0.74     27695
   macro avg       0.74      0.74      0.74     27695
weighted avg       0.74      0.74      0.74     27695



In [ ]:
for doc, prediction, y in zip(X_test[0:20], predictions_NN[0:20], y_test[0:20]):
    print('%r => %s [%s]' % (doc, prediction, y))

'nunc peg ranç tant gent famíl rápid ness ele ' => Negativo [Positivo]
'supost pesquis ? tant quant sab veríd ach legal invert tud manch candidat link mesm coloc desm fal víde menin ' => Positivo [Positivo]
'áudi dela reforç tes odebrecht deu dinh amig tem diz pf lei transcr ' => Neutro [Neutro]
'escolh deput barr projet presid ? khkjkjkjk tá difícil polít ' => Positivo [Negativo]
'conf seleç tapet dar vid complement ambi cas ' => Neutro [Neutro]
'marin fal alg ? vi ' => Negativo [Negativo]
'‘ fal beste sim moment raiv separ ’, diz ex mulh bolsonar ' => Positivo [Neutro]
'vot vam vot ' => Positivo [Positivo]
'candidat ? — vot tô tailând ' => Negativo [Negativo]
'nezer pûç be ' => Positivo [Positivo]
'exat entend ministr fazend vai ser paul gued ach lul entend econom ? kkk ' => Positivo [Positivo]
'profes liset candidat psol govern sp defend discuss sobr gêner escol promet ajud municípi constru crech ' => Neutro [Neutro]
'grand culp aind sequ eleit sequ extingu orgã inútil dé dinh sufic

## SVM model

In [ ]:
model_SVC = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC(random_state=42)),
])

## SVM model with Hold out

In [ ]:
model_SVC.fit(X_train, y_train)
predictions_SVC = model_SVC.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions_SVC))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



              precision    recall  f1-score   support

    Negativo       0.00      0.00      0.00      9813
      Neutro       0.33      1.00      0.50      9664
    Positivo       0.00      0.00      0.00      9813

    accuracy                           0.33     29290
   macro avg       0.11      0.33      0.17     29290
weighted avg       0.11      0.33      0.16     29290



In [ ]:
for doc, prediction, y in zip(X_test[0:20], predictions_SVC[0:20], y_test[0:20]):
    print('%r => %s [%s]' % (doc, prediction, y))

'saib nazist ode jud jerusalém caracterís esquerdopat . além diss esquerd ador corrupt , pedófil , estupr marg . enquadr ness tim ? pesquis amig lulal merd mesm cois :-) ' => Positivo [Positivo]
'príncip harry complet 34 ano receb homen intern ' => Neutro [Neutro]
'blizzard problem overwatch , diz licenç jog tav jog ate ont boa , tod bug compr jog mó feliz mei deix entr ' => Positivo [Negativo]
'oi mig ' => Positivo [Positivo]
'imagin iri ter gent centr quer ir pro lad bolsonar . ' => Negativo [Positivo]
'ali marin tem alv dobr bolsonar daciol debat . ' => Negativo [Neutro]
'aha !!! import pro irm irm vai sent bem famíl lá ^^ ' => Positivo [Positivo]
'big broth ingl políci vê 11 mil hor víde ach suspeit envenen ex espiã . ' => Neutro [Neutro]
'agor sim united par cicl posi watford som prim derrot nest ediç premi leag ' => Positivo [Positivo]
'post elogi manu aqu . ler post , vai entend toler ser zer pra tip açã . ele terr . candidat pres . atitud infeliz . :-( ' => Negativo [Negativo]


## Logistic Regression

In [ ]:
model_LR = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression(random_state=42, multi_class='multinomial', max_iter=500, n_jobs=-1)),
])

## Logistic Regression with Hold Out

In [ ]:
model_LR.fit(X_train, y_train)
predictions_LR = model_LR.predict(X_test)

In [ ]:
X_train.shape, X_test.shape

((56232,), (27697,))

In [ ]:
print(classification_report(y_test, predictions_LR))

              precision    recall  f1-score   support

    Negativo       0.73      0.78      0.75      9254
      Neutro       0.86      0.86      0.86      9125
    Positivo       0.70      0.66      0.68      9318

    accuracy                           0.76     27697
   macro avg       0.76      0.76      0.76     27697
weighted avg       0.76      0.76      0.76     27697



In [ ]:
for doc, prediction, y in zip(X_test[0:20], predictions_LR[0:20], y_test[0:20]):
    print('%r => %s [%s]' % (doc, prediction, y))

'incomod idolatr exempl pi tortur histór país mim incomod diz sobr além outr pequen cois ”. val difer bom vot ' => Positivo [Positivo]
'explic motiv porqu project metr mondeg avanç apes milhã gast afinal macumb "... segund géni presid municípi coimbr ' => Neutro [Positivo]
'extr meirell gast r 21 milhã pont bolsonar r 39 mil ' => Neutro [Neutro]
'cir man ' => Negativo [Negativo]
'fiscal encontr 1 6 mil médic registr red hospital rio ' => Neutro [Neutro]
'ju candidat c apo ódi coraçã ' => Negativo [Negativo]
'franç evit declar apoi bolsonar haddad afin nenhum ' => Negativo [Neutro]
'regrann from marin convers desenrol fio vid sempr vem sempr gost compartilh vid ! ' => Positivo [Positivo]
'tá cad dia difícil segu pesso facebook princip express opin polít ' => Negativo [Negativo]
'ótim ' => Positivo [Positivo]
'i liked vide cs go ! astralil vs faz melhor jog dia ! ' => Positivo [Positivo]
'charli sheen ped reduç pens filh ' => Neutro [Neutro]
'acredit cir vai vir aqu cidad consegu ver ' =

## Logistic Regression with cross-validation and gridsearch 
Doing CV and gridsearch on the model with the best performance 

In [ ]:
model_LR = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression(random_state=42, multi_class='multinomial', max_iter=500, n_jobs=-1)),
])

In [ ]:
param_grid_LR = {
    'clf__solver': ['newton-cg', 'lbfgs'],
    'clf__penalty': ['l2', 'none'],
}

In [ ]:
grid_pipeline_LR = GridSearchCV(param_grid=param_grid_LR, estimator=model_LR, scoring='accuracy', cv=5)

In [ ]:
grid_pipeline_LR.fit(X_train, y_train)

In [ ]:
print(grid_pipeline_LR.best_params_)

{'clf__penalty': 'l2', 'clf__solver': 'lbfgs'}


In [ ]:
predictions_LR = grid_pipeline_LR.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions_LR))

              precision    recall  f1-score   support

    Negativo       0.73      0.77      0.75      9256
      Neutro       0.86      0.86      0.86      9128
    Positivo       0.70      0.66      0.68      9313

    accuracy                           0.76     27697
   macro avg       0.76      0.76      0.76     27697
weighted avg       0.76      0.76      0.76     27697



In [ ]:
for doc, prediction, y in zip(X_test[0:20], predictions_LR[0:20], y_test[0:20]):
    print('%r => %s [%s]' % (doc, prediction, y))

'um outr problem nov tip model ele faz pont mud opin temp temp vá pra opin maior vi ! aí gent pod estim dev venc ! alô ' => Positivo [Positivo]
'menos quer tant ir nunc oportun tant ódi dess govern podr verdad perd esperanç ness país ' => Negativo [Negativo]
'ele 2018 2014 saib mud ' => Neutro [Neutro]
'7 10 11 07 thierry eaí bru fal tant polít vot ? 7 10 11 09 brunn kkkkkkkklkemoticon_neg 7 10 11 09 thierry doid ? tô fal séri 7 10 11 11 brunn vot bro ' => Positivo [Negativo]
'skaf cham dor polít quint categor ' => Neutro [Neutro]
'emocion entrev cir marin ' => Negativo [Negativo]
'cir gom seg risc orient pass imag tranquil ' => Positivo [Neutro]
'0 minut ler alg sobr polít timelin record 3 minut obrig jov nerd ' => Positivo [Positivo]
'ah velh tad cir ' => Negativo [Negativo]
'bel ' => Positivo [Positivo]
'nad melhor cheg cas ter var mulh grit bolsonar frent condomini vari carr polic ' => Positivo [Positivo]
'googl lanç serviç transmiss onlin jog ' => Neutro [Neutro]
'nunc cir smp vai

## Exporting the model pipeline

In [ ]:
import pickle

In [ ]:
pickle.dump(grid_pipeline_LR, open('pipeline_LR', 'wb'))

In [ ]:
loaded_model = pickle.load(open('pipeline_LR', 'rb'))

In [ ]:
predictions = loaded_model.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

    Negativo       0.73      0.78      0.75      9254
      Neutro       0.86      0.86      0.86      9125
    Positivo       0.70      0.66      0.68      9318

    accuracy                           0.76     27697
   macro avg       0.76      0.76      0.76     27697
weighted avg       0.76      0.76      0.76     27697

